In [1]:
from utils import max_features, maxlen, embed_size_fastText, embed_size_glove, embed_size_glove_twitter
from utils import earlystop, batch_size, epochs, run_5fold, write_prediction
from model import get_textRCNN_model
import pickle
import tensorflow as tf
import pandas as pd

/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/home/ec2-user/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/home/ec2-user/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))
/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into whic

In [2]:
fastText = pickle.load(open("input/fastText.300d.pickle", "rb"))
glove = pickle.load(open("input/glove.300d.pickle", "rb"))
glove_twitter = pickle.load(open("input/glove.twitter.200d.pickle", "rb"))

In [3]:
x_train = pickle.load(open("input/x_train.pickle", "rb"))
y_train = pickle.load(open("input/y_train.pickle", "rb"))
x_test = pickle.load(open("input/x_test.pickle", "rb"))
submission = pd.read_csv("input/sample_submission.csv")

## check different embedding

In [4]:
params = {
    "maxlen": maxlen,
    "max_features": max_features,
    "num_filter": 50,
    "reg": 0.0,
    "dropout": 0.2,
    "units": 64,
    "kernel_size": 3
}

In [5]:
embeddings = [(fastText, embed_size_fastText, "fastText"), (glove, embed_size_glove, "glove"), (glove_twitter, embed_size_glove_twitter, "glove_twitter")]

In [6]:
predictions = {}
for embed in embeddings:
    print ("\n********************{}********************".format(embed[2]))
    params["embedding_matrix"] = embed[0]
    params["embed_size"] = embed[1]
    y_preds = run_5fold(x_train, y_train, x_test, get_textRCNN_model, **params)
    predictions[embed[2]] = y_preds


********************fastText********************
Instructions for updating:
Please switch to tf.metrics.auc. Note that the order of the labels and predictions arguments has been switched.
Train on 127656 samples, validate on 31915 samples
Epoch 1/50
 - 1163s - loss: 0.0589 - acc: 0.9795 - auc_roc: 0.9245 - val_loss: 0.0434 - val_acc: 0.9828 - val_auc_roc: 0.9747
Epoch 2/50


KeyboardInterrupt: 

##### result: 
- fastText: 



- glove: 



- glove.twitter: 




## check num_filter
(suggest values: 100~600: test 100, 300, 500)

In [ ]:
filters = [100, 300, 500]  # for the result of 50, refer to the previous log

In [ ]:
params = {
    "maxlen": maxlen,
    "max_features": max_features,
    "reg": 0.0,
    "dropout": 0.2,
    "embedding_matrix": fastText,
    "embed_size": embed_size_fastText
}

In [ ]:
predictions = {}
for f in filters:
    print ("\n********************f = {}********************".format(f))
    params["num_filter"] = f
    y_preds = run_5fold(x_train, y_train, x_test, get_textRCNN_model, **params)
    predictions[f] = y_preds

###### result: 

- f = 50


- f = 100



- f = 300



- f = 500



In [ ]:
optimal_pred = predictions[50]

## check kernel_size
(suggest value: [3, 5, 7, 9])

In [ ]:
params = {
    "maxlen": maxlen,
    "max_features": max_features,
    "dropout": 0.2,
    "num_filter": 50,
    "reg": 0.0,
    "embedding_matrix": fastText,
    "embed_size": embed_size_fastText
}

In [ ]:
kernels = [5, 7, 9]  # for 3, see result above

In [ ]:
predictions = {}
for k in kernels:
    print ("\n********************k = {}********************".format(k))
    params["kernel_size"] = k
    y_preds = run_5fold(x_train, y_train, x_test, get_textRCNN_model, **params)
    predictions[k] = y_preds

##### result: 

- k = 3




- k = 5



- k = 7



- k = 9



In [ ]:
optimal_pred = predictions[9]

## check drop out
(suggest value 0.1~0.5: test 0.1, 0.2, 0.5)

In [ ]:
params = {
    "maxlen": maxlen,
    "max_features": max_features,
    "kernel_size": 9,
    "num_filter": 50,
    "reg": 0.0,
    "embedding_matrix": fastText,
    "embed_size": embed_size_fastText
}

In [ ]:
dropouts = [0.1, 0.5] # 0.2 see previous result

In [ ]:
predictions = {}
for d in dropouts:
    print ("\n********************d = {}********************".format(d))
    params["dropout"] = d
    y_preds = run_5fold(x_train, y_train, x_test, get_textRCNN_model, **params)
    predictions[d] = y_preds

##### result

- d = 0.1



- d = 0.2



- d = 0.5



In [ ]:
optimal_pred = 

## check different units
(64, 128)

#### tuned params

In [ ]:
params_textCNN = {
    "maxlen": maxlen,
    "max_features": max_features,
    "embedding_size":,
    "embed_matrix":,
    "num_filter": 300,
    "reg": 0.0,
    "dropout": 0.2
}

### write result

In [ ]:
write_prediction(optimal_pred, submission, "output/textCNN.csv")